In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  7 22:08:46 2019

@author: Deechean
"""
from cifar10 import cifar10
import os
import tensorflow as tf
from lenet5 import LetNet5
import tf_general as tfg
import numpy as np
import aws_boto3

FLAGS = tf.flags.FLAGS

try:
    tf.flags.DEFINE_string('f', '', 'kernel')
    tf.flags.DEFINE_integer('epoch', 10000, 'epoch')
    tf.flags.DEFINE_integer('batch_size',100, 'batch size')
    tf.flags.DEFINE_integer('test_size', 100, 'test size')
    tf.flags.DEFINE_float('lr', 0.05, 'learning rate')
    tf.flags.DEFINE_float('ckpt_frequency', 250, 'frequency to save checkpoint')
    tf.flags.DEFINE_boolean('restore', False, 'restore from checkpoint and run test')
    print('parameters were defined.')
except:
    print('parameters have been defined.')

#CONTINUE = 0

parameters were defined.


In [2]:
data = cifar10();
train_image, train_label,_ = data.get_train_batch(FLAGS.batch_size)

train set length: 50000
test image shape: (10000, 32, 32, 3)
test label shape: (10000,)
test set length: 10000


In [3]:
ckpt_dir = 'ckpt/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

In [4]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 28,28,3], name='x_input')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    y_ = tf.placeholder(tf.int64, [None], name='labels')

In [5]:
with tf.name_scope('prediction'):
    le_net5 = LetNet5(x, keep_prob)
    y = le_net5.prediction

Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                                                  labels=y_, 
                                                                                  name="cross_entropy_per_example"))

In [7]:
with tf.name_scope('train_step'):
    train_step = tf.train.AdagradOptimizer(FLAGS.lr).minimize(cross_entropy)

In [8]:
prediction =tf.argmax(y, 1)
accuracy = tf.reduce_mean(tf.cast( tf.equal(prediction,y_), tf.float32))

In [9]:
def SaveCheckpoint2S3():
    bucketname = 'sagemaker-deechean-dl' 
    path = 'lenet_ckpt/'
    for file in os.listdir('ckpt'):
        if os.path.isfile('ckpt/'+file):
            aws_boto3.upload_file('ckpt/'+file, bucketname,path+file)

In [10]:
import time

if __name__ == '__main__':
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(max_to_keep = 1)
        for i in range(FLAGS.epoch):
            train_image, train_label,_ = data.get_train_batch(FLAGS.batch_size)
            _,accuracy_rate = sess.run([train_step,accuracy], feed_dict={keep_prob: 0.5, x:train_image, y_:train_label})
            tfg.savelog('accuracy',i, accuracy_rate)          
            if (i+1) % FLAGS.ckpt_frequency == 0:  #保存预测模型
                saver.save(sess,'ckpt/cifar10_'+str(i+1)+'.ckpt',global_step=i)
                SaveCheckpoint2S3()
                acc_accuracy = 0
                for j in range(int(10000/FLAGS.test_size)):                    
                    test_image, test_label,test_index = data.get_test_batch(FLAGS.test_size)
                    accuracy_rate, output = sess.run([accuracy,prediction],feed_dict={keep_prob: 1, x:test_image, y_:test_label})
                    acc_accuracy += accuracy_rate
                    tfg.savelog('test_index',i, test_index)
                    tfg.savelog('output',i, output)
                accuracy_rate = acc_accuracy/10000*FLAGS.test_size
                print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' iter ' + str(i) + ', Test accuracy:' +str(round(accuracy_rate*100,2))+'%')
                tfg.savelog('test_accuracy',i, accuracy_rate)            
    tf.reset_default_graph()

2019-06-02 02:01:24 iter 249, Test accuracy:29.2%
Instructions for updating:
Use standard file APIs to delete files with this prefix.
2019-06-02 02:02:16 iter 499, Test accuracy:30.73%


KeyboardInterrupt: 